In [3]:
from model_builders import *

# Create a Myronenko Encoder
encoder = ResnetEncoder(input_nc=3,
                        output_nc=3,
                        ngf=64,
                        n_blocks=6,
                        norm_layer=nn.BatchNorm3d,
                        use_dropout=False,
                        padding_type='reflect')
print(encoder)

ResnetEncoder(
  (model): Sequential(
    (0): ReflectionPad3d((3, 3, 3, 3, 3, 3))
    (1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), bias=False)
    (2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
    (5): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
    (8): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): ResnetBlock(
      (res_block): Sequential(
        (0): ReflectionPad3d((1, 1, 1, 1, 1, 1))
        (1): Conv3d(256, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1), bias=False)
        (2): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): 

In [21]:
import nibabel as nib

# Path to an image
path = "C:\\tractography\\DWI_concatenated_resized_b0.nii.gz"

# Load the image
image = nib.load(path)

# Get the data
data = image.get_fdata()

print(data.shape)

data_torch = torch.from_numpy(data).unsqueeze(0).unsqueeze(0).float()

print(data_torch.shape)

(128, 178, 115)
torch.Size([1, 1, 128, 178, 115])


In [56]:
model = []

input_nc = 1
ngf = 64
norm_layer = nn.BatchNorm3d


padding_layer = nn.ReflectionPad3d(3)
model += [padding_layer]
model += [nn.Conv3d(input_nc, ngf, kernel_size=7, padding=0, bias=False), 
                norm_layer(ngf), nn.ReLU(True)]

# 2. Add the downsample block
number_downsampling = 2
for i in range(number_downsampling):
    mult = 2**i
    model += [nn.Conv3d(ngf * mult, ngf * mult * 2, kernel_size=3, 
                        stride=2, padding=1, bias=False), 
                    norm_layer(ngf * mult * 2), 
                    nn.ReLU(True)]
    
mult = 2**number_downsampling
n_blocks = 6
for i in range(n_blocks):
    model += [ResnetBlock(ngf * mult, padding_type="replicate", 
                            norm_layer=norm_layer, use_dropout=False, 
                            use_bias=False)]

for i in range(number_downsampling):
    mult = 2**(number_downsampling - i)
    model += [nn.Conv3d(ngf * mult, int(ngf * mult / 2), 
                        kernel_size=3, stride=2, padding=1, bias=False), 
                    norm_layer(int(ngf * mult / 2)), 
                    nn.ReLU(True)]
    
model += [nn.Conv3d(int(ngf * mult / 2), int(ngf * mult / 4),
                    kernel_size=3, stride=2, padding=1, bias=False),
                        norm_layer(int(ngf * mult / 4)), nn.ReLU(True)]

model += [nn.Conv3d(int(ngf * mult / 4), 1, kernel_size=3, stride=2, padding=1, bias=False)]

model += [nn.AdaptiveAvgPool3d((1, 1, 1))]

In [61]:
def forward(model, output):
    for layer in model:
        output = layer(output)
        print(output.shape)
    return output

# for layer in model:
#     output = model_final(torch.from_numpy(data).unsqueeze(0).unsqueeze(0).float())
output = forward(model, torch.from_numpy(data).unsqueeze(0).unsqueeze(0).float())

torch.Size([1, 1, 134, 184, 121])
torch.Size([1, 64, 128, 178, 115])
torch.Size([1, 64, 128, 178, 115])
torch.Size([1, 64, 128, 178, 115])
torch.Size([1, 128, 64, 89, 58])
torch.Size([1, 128, 64, 89, 58])
torch.Size([1, 128, 64, 89, 58])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 256, 32, 45, 29])
torch.Size([1, 128, 16, 23, 15])
torch.Size([1, 128, 16, 23, 15])
torch.Size([1, 128, 16, 23, 15])
torch.Size([1, 64, 8, 12, 8])
torch.Size([1, 64, 8, 12, 8])
torch.Size([1, 64, 8, 12, 8])
torch.Size([1, 32, 4, 6, 4])
torch.Size([1, 32, 4, 6, 4])
torch.Size([1, 32, 4, 6, 4])
torch.Size([1, 1, 2, 3, 2])
torch.Size([1, 1, 1, 1, 1])


In [62]:
# Squeeze the output
output = output.squeeze(0).squeeze(0).detach().numpy()

In [63]:
print(data.shape)
print(output.shape)
print(output)

(128, 178, 115)
(1, 1, 1)
[[[0.20566858]]]
